In [402]:
import pandas as pd
import sqlite3 as sq
import matplotlib as mpl
from matplotlib import rcParams
import matplotlib.pyplot as plt
import numpy as np
import requests
pd.set_option('display.max_rows', 1000); pd.set_option('display.max_columns', 1000); pd.set_option('display.width', 1000)
pd.options.mode.chained_assignment = None
import pickle
import sqlite3 as sq

In [477]:
conn = sq.connect('../../Data-Pipelines/Outputs/CensusBureau.db')
sql_query = pd.read_sql('SELECT * FROM [CensusBureau_ACS5_2022_1]', conn)
ACS52022_1 = pd.DataFrame(sql_query)
sql_query = pd.read_sql('SELECT * FROM [CensusBureau_ACS5_2022_2]', conn)
ACS52022_2 = pd.DataFrame(sql_query)
ACS52022 = ACS52022_1.merge(ACS52022_2, on = ['NAME', 'GEO_ID', 'Source'])
ACS52022.drop(columns = 'Source', inplace = True)

In [478]:
avg = ACS52022[['NAME', 'Average Household Size']]
thelist = ['Cheatham County, Tennessee', 'Davidson County, Tennessee', 'Dickson County, Tennessee', 'Houston County, Tennessee', 
           'Humphreys County, Tennessee', 'Maury County, Tennessee', 'Montgomery County, Tennessee', 'Robertson County, Tennessee', 
           'Rutherford County, Tennessee', 'Stewart County, Tennessee', 'Sumner County, Tennessee', 'Trousdale County, Tennessee', 
           'Williamson County, Tennessee', 'Wilson County, Tennessee']
avg = avg.loc[avg['NAME'].isin(thelist)].reset_index(drop = True)
avg = avg.rename(columns = {'Average Household Size': '20225YR'})
avg.head()

,NAME,20225YR
0,"Cheatham County, Tennessee",2.60
1,"Davidson County, Tennessee",2.28
2,"Dickson County, Tennessee",2.65
3,"Houston County, Tennessee",2.66
4,"Humphreys County, Tennessee",2.79


In [479]:
conn = sq.connect('../../Data-Pipelines/Outputs/WoodsandPooleandAffiliated.db')
sql_query = pd.read_sql('SELECT * FROM [WPHouseholds_2023Base]', conn)
hhsize = pd.DataFrame(sql_query)
hhsize = hhsize.loc[hhsize['Year'] != 'None']

In [480]:
hhsize = hhsize[['NAME', 'Year', 'personsperhousehold']]
hhsize['Year'] = hhsize['Year'].astype(int)
hhsize = hhsize[hhsize['Year'] > 2021]
hhsize['Year'] = hhsize['Year'].astype(str)

In [481]:
hhsize.head(2)

,NAME,Year,personsperhousehold
32,"Cheatham County, Tennessee",2022,2.47
33,"Cheatham County, Tennessee",2023,2.46


In [482]:
wide_df = hhsize.pivot_table(index='NAME', columns='Year', values='personsperhousehold')

In [483]:
wide_df = wide_df.reset_index().rename_axis(None, axis=1)

In [484]:
wide_df.head(2)

,NAME,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050,2051,2052,2053,2054,2055,2056,2057,2058,2059,2060
0,"Cheatham County, Tennessee",2.47,2.46,2.45,2.45,2.44,2.44,2.43,2.43,2.43,2.43,2.43,2.43,2.43,2.43,2.43,2.43,2.43,2.44,2.44,2.44,2.44,2.44,2.44,2.44,2.44,2.44,2.44,2.44,2.44,2.43,2.43,2.43,2.43,2.43,2.42,2.42,2.42,2.42,2.42
1,"Davidson County, Tennessee",2.27,2.26,2.25,2.24,2.24,2.23,2.23,2.23,2.23,2.23,2.23,2.23,2.23,2.23,2.24,2.24,2.24,2.24,2.24,2.25,2.25,2.25,2.25,2.25,2.25,2.25,2.25,2.25,2.25,2.25,2.25,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24


In [485]:
wide_df = wide_df.rename(columns = {'2022': '2022 WP'})

In [486]:
merged_df = pd.merge(wide_df, avg[['NAME', '20225YR']], on = 'NAME', how = 'inner')
column_order = ['NAME', '20225YR'] + [col for col in merged_df.columns if col not in ['NAME', '20225YR']]
merged_df = merged_df[column_order]

In [487]:
non_adj = merged_df
non_adj = non_adj.drop(columns = ['20225YR', '2022 WP']).set_index('NAME')
non_adj = non_adj.add_suffix(" Non-Adjusted").reset_index()

In [488]:
years = list(merged_df.columns)
years.remove('NAME')
years.remove('20225YR')
years.remove('2022 WP')
years.remove('2050')

In [489]:
def apply_formula(row):
    # Assuming '2022' is the base year for census and WP, and '2050' is the last year
    base_year = 2022
    final_year = 2050

    # Calculate the denominator part of the formula
    denominator = final_year - base_year
    
    # If denominator is zero, return 0 to avoid division by zero error
    if denominator == 0:
        return 0
    # Loop through the years between 2024 and 2049 (inclusive)
    for year in range(base_year + 1, final_year):
        # Calculate the adjusted value for the current year
        adjusted_value = row[f'{year}'] - (row[f'{base_year} WP'] - row[f'{base_year}5YR']) / denominator * (final_year - int(year))
       # Update the value for the current year
        row[f'{year}'] = adjusted_value
    
    return row

# Apply the formula to each row in the DataFrame
merged_df = merged_df.apply(apply_formula, axis=1)

In [490]:
data = merged_df.merge(non_adj, on = 'NAME')

In [491]:
data.head()

,NAME,20225YR,2022 WP,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050,2051,2052,2053,2054,2055,2056,2057,2058,2059,2060,2023 Non-Adjusted,2024 Non-Adjusted,2025 Non-Adjusted,2026 Non-Adjusted,2027 Non-Adjusted,2028 Non-Adjusted,2029 Non-Adjusted,2030 Non-Adjusted,2031 Non-Adjusted,2032 Non-Adjusted,2033 Non-Adjusted,2034 Non-Adjusted,2035 Non-Adjusted,2036 Non-Adjusted,2037 Non-Adjusted,2038 Non-Adjusted,2039 Non-Adjusted,2040 Non-Adjusted,2041 Non-Adjusted,2042 Non-Adjusted,2043 Non-Adjusted,2044 Non-Adjusted,2045 Non-Adjusted,2046 Non-Adjusted,2047 Non-Adjusted,2048 Non-Adjusted,2049 Non-Adjusted,2050 Non-Adjusted,2051 Non-Adjusted,2052 Non-Adjusted,2053 Non-Adjusted,2054 Non-Adjusted,2055 Non-Adjusted,2056 Non-Adjusted,2057 Non-Adjusted,2058 Non-Adjusted,2059 Non-Adjusted,2060 Non-Adjusted
0,"Cheatham County, Tennessee",2.60,2.47,2.585357,2.570714,2.566071,2.551429,2.546786,2.532143,2.5275,2.522857,2.518214,2.513571,2.508929,2.504286,2.499643,2.495,2.490357,2.485714,2.491071,2.486429,2.481786,2.477143,2.4725,2.467857,2.463214,2.458571,2.453929,2.449286,2.444643,2.44,2.43,2.43,2.43,2.43,2.43,2.42,2.42,2.42,2.42,2.42,2.46,2.45,2.45,2.44,2.44,2.43,2.43,2.43,2.43,2.43,2.43,2.43,2.43,2.43,2.43,2.43,2.44,2.44,2.44,2.44,2.44,2.44,2.44,2.44,2.44,2.44,2.44,2.44,2.43,2.43,2.43,2.43,2.43,2.42,2.42,2.42,2.42,2.42
1,"Davidson County, Tennessee",2.28,2.27,2.269643,2.259286,2.248929,2.248571,2.238214,2.237857,2.2375,2.237143,2.236786,2.236429,2.236071,2.235714,2.235357,2.245,2.244643,2.244286,2.243929,2.243571,2.253214,2.252857,2.2525,2.252143,2.251786,2.251429,2.251071,2.250714,2.250357,2.25,2.25,2.25,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.26,2.25,2.24,2.24,2.23,2.23,2.23,2.23,2.23,2.23,2.23,2.23,2.23,2.24,2.24,2.24,2.24,2.24,2.25,2.25,2.25,2.25,2.25,2.25,2.25,2.25,2.25,2.25,2.25,2.25,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24
2,"Dickson County, Tennessee",2.65,2.47,2.633571,2.617143,2.600714,2.594286,2.577857,2.571429,2.5650,2.558571,2.552143,2.545714,2.539286,2.532857,2.526429,2.520,2.513571,2.507143,2.510714,2.504286,2.497857,2.491429,2.4850,2.478571,2.472143,2.465714,2.459286,2.452857,2.446429,2.44,2.43,2.43,2.43,2.43,2.43,2.42,2.42,2.42,2.42,2.42,2.46,2.45,2.44,2.44,2.43,2.43,2.43,2.43,2.43,2.43,2.43,2.43,2.43,2.43,2.43,2.43,2.44,2.44,2.44,2.44,2.44,2.44,2.44,2.44,2.44,2.44,2.44,2.44,2.43,2.43,2.43,2.43,2.43,2.42,2.42,2.42,2.42,2.42
3,"Houston County, Tennessee",2.66,2.26,2.635714,2.611429,2.597143,2.572857,2.558571,2.544286,2.5300,2.515714,2.491429,2.487143,2.472857,2.458571,2.444286,2.430,2.415714,2.401429,2.397143,2.382857,2.368571,2.354286,2.3500,2.335714,2.321429,2.307143,2.292857,2.278571,2.264286,2.25,2.25,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.25,2.24,2.24,2.23,2.23,2.23,2.23,2.23,2.22,2.23,2.23,2.23,2.23,2.23,2.23,2.23,2.24,2.24,2.24,2.24,2.25,2.25,2.25,2.25,2.25,2.25,2.25,2.25,2.25,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24
4,"Humphreys County, Tennessee",2.79,2.31,2.752857,2.735714,2.708571,2.691429,2.664286,2.647143,2.6300,2.612857,2.595714,2.578571,2.561429,2.544286,2.527143,2.520,2.502857,2.485714,2.468571,2.451429,2.444286,2.427143,2.4100,2.392857,2.375714,2.358571,2.341429,2.324286,2.307143,2.29,2.29,2.29,2.29,2.28,2.28,2.28,2.28,2.28,2.28,2.28,2.29,2.29,2.28,2.28,2.27,2.27,2.27,2.27,2.27,2.27,2.27,2.27,2.27,2.28,2.28,2.28,2.28,2.28,2.29,2.29,2.29,2.29,2.29,2.29,2.29,2.29,2.29,2.29,2.29,2.29,2.29,2.28,2.28,2.28,2.28,2.28,2.28,2.28


In [492]:
data.to_csv('../data/smoothedavghhsize_20182022ACS.csv', index = False)

In [493]:
gq = {'Cheatham County, Tennessee': 330, 'Davidson County, Tennessee': 23732, 'Dickson County, Tennessee': 767, 'Houston County, Tennessee': 131, 
      'Humphreys County, Tennessee': 425, 'Maury County, Tennessee': 1267, 'Montgomery County, Tennessee': 4446, 'Robertson County, Tennessee': 641, 
      'Rutherford County, Tennessee': 5338, 'Stewart County, Tennessee': 203, 'Sumner County, Tennessee': 1492, 'Trousdale County, Tennessee': 2512, 
      'Williamson County, Tennessee': 1265, 'Wilson County, Tennessee': 1708}
hh = {'Cheatham County, Tennessee': 16125, 'Davidson County, Tennessee': 327906, 'Dickson County, Tennessee': 21118, 'Houston County, Tennessee': 3106, 
      'Humphreys County, Tennessee': 6831, 'Maury County, Tennessee': 43450, 'Montgomery County, Tennessee': 91250, 'Robertson County, Tennessee': 29510, 
      'Rutherford County, Tennessee': 134986, 'Stewart County, Tennessee': 5173, 'Sumner County, Tennessee': 80981, 'Trousdale County, Tennessee': 3783, 
      'Williamson County, Tennessee': 95016, 'Wilson County, Tennessee': 62778}
hhpop =  {'Cheatham County, Tennessee': 41924, 'Davidson County, Tennessee': 688602, 'Dickson County, Tennessee': 55962, 'Houston County, Tennessee': 8262, 
      'Humphreys County, Tennessee': 18784, 'Maury County, Tennessee': 109493, 'Montgomery County, Tennessee': 235426, 'Robertson County, Tennessee': 76135, 
      'Rutherford County, Tennessee': 361763, 'Stewart County, Tennessee': 14019, 'Sumner County, Tennessee': 206502, 'Trousdale County, Tennessee': 9759, 
      'Williamson County, Tennessee': 263195, 'Wilson County, Tennessee': 161966}

In [494]:
hhbase = pd.DataFrame(list(hh.items()), columns=['NAME', '2023 Census'])
hhbase.head()

,NAME,2023 Census
0,"Cheatham County, Tennessee",16125
1,"Davidson County, Tennessee",327906
2,"Dickson County, Tennessee",21118
3,"Houston County, Tennessee",3106
4,"Humphreys County, Tennessee",6831


In [495]:
conn = sq.connect('../../Data-Pipelines/Outputs/WoodsandPooleandAffiliated.db')
sql_query = pd.read_sql('SELECT * FROM [WPHouseholds_2023Base]', conn)
hhs = pd.DataFrame(sql_query)
hhs = hhs.loc[hhs['Year'] != 'None']

In [496]:
hhs = hhs[['NAME', 'Year', 'households']]
hhs['Year'] = hhs['Year'].astype(int)
hhs = hhs[hhs['Year'] > 2022]
hhs['Year'] = hhs['Year'].astype(str)

In [497]:
wide_df = hhs.pivot_table(index='NAME', columns='Year', values='households')
wide_df = wide_df.reset_index().rename_axis(None, axis=1)
wide_df = wide_df.rename(columns = {'2023': '2023 WP'})

In [498]:
merged_df = pd.merge(wide_df, hhbase[['NAME', '2023 Census']], on = 'NAME', how = 'inner')
column_order = ['NAME', '2023 Census'] + [col for col in merged_df.columns if col not in ['NAME', '2023 Census']]
merged_df = merged_df[column_order]

In [499]:
non_adj = merged_df
non_adj = non_adj.drop(columns = ['2023 Census', '2023 WP']).set_index('NAME')
non_adj = non_adj.add_suffix(" Non-Adjusted").reset_index()

In [500]:
years = list(merged_df.columns)
years.remove('NAME')
years.remove('2023 Census')
years.remove('2023 WP')
years.remove('2050')

In [501]:
def apply_formula(row):
    # Assuming '2023' is the base year for census and WP, and '2050' is the last year
    base_year = 2023
    final_year = 2050

    # Calculate the denominator part of the formula
    denominator = final_year - base_year
    
    # If denominator is zero, return 0 to avoid division by zero error
    if denominator == 0:
        return 0
    # Loop through the years between 2024 and 2049 (inclusive)
    for year in range(base_year + 1, final_year):
        # Calculate the adjusted value for the current year
        adjusted_value = row[f'{year}'] - (row[f'{base_year} WP'] - row[f'{base_year} Census']) / denominator * (final_year - int(year))
       # Update the value for the current year
        row[f'{year}'] = adjusted_value
    
    return row

# Apply the formula to each row in the DataFrame
merged_df = merged_df.apply(apply_formula, axis=1)

In [502]:
data = merged_df.merge(non_adj, on = 'NAME')

In [503]:
data.head()

,NAME,2023 Census,2023 WP,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050,2051,2052,2053,2054,2055,2056,2057,2058,2059,2060,2024 Non-Adjusted,2025 Non-Adjusted,2026 Non-Adjusted,2027 Non-Adjusted,2028 Non-Adjusted,2029 Non-Adjusted,2030 Non-Adjusted,2031 Non-Adjusted,2032 Non-Adjusted,2033 Non-Adjusted,2034 Non-Adjusted,2035 Non-Adjusted,2036 Non-Adjusted,2037 Non-Adjusted,2038 Non-Adjusted,2039 Non-Adjusted,2040 Non-Adjusted,2041 Non-Adjusted,2042 Non-Adjusted,2043 Non-Adjusted,2044 Non-Adjusted,2045 Non-Adjusted,2046 Non-Adjusted,2047 Non-Adjusted,2048 Non-Adjusted,2049 Non-Adjusted,2050 Non-Adjusted,2051 Non-Adjusted,2052 Non-Adjusted,2053 Non-Adjusted,2054 Non-Adjusted,2055 Non-Adjusted,2056 Non-Adjusted,2057 Non-Adjusted,2058 Non-Adjusted,2059 Non-Adjusted,2060 Non-Adjusted
0,"Cheatham County, Tennessee",16125,16950.0,16314.555556,16493.111111,16662.666667,16825.222222,16981.777778,17130.333333,17271.888889,17408.444444,17540.000000,17669.555556,17796.111111,17921.666667,18046.222222,18171.777778,18297.333333,18419.888889,18544.444444,18673.000000,18804.555556,18941.111111,19080.666667,19221.222222,19362.777778,19507.333333,19656.888889,19811.444444,19969.0,20099.0,20231.0,20365.0,20499.0,20630.0,20759.0,20886.0,21010.0,21133.0,21252.0,17109.0,17257.0,17396.0,17528.0,17654.0,17772.0,17883.0,17989.0,18090.0,18189.0,18285.0,18380.0,18474.0,18569.0,18664.0,18756.0,18850.0,18948.0,19049.0,19155.0,19264.0,19374.0,19485.0,19599.0,19718.0,19842.0,19969.0,20099.0,20231.0,20365.0,20499.0,20630.0,20759.0,20886.0,21010.0,21133.0,21252.0
1,"Davidson County, Tennessee",327906,304315.0,330101.259259,332050.518519,333789.777778,335346.037037,336726.296296,337908.555556,338917.814815,339788.074074,340511.333333,341116.592593,341623.851852,342033.111111,342367.370370,342633.629630,342827.888889,342945.148148,343027.407407,343134.666667,343297.925926,343502.185185,343731.444444,343946.703704,344168.962963,344449.222222,344816.481481,345265.740741,345767.0,347216.0,348706.0,350230.0,351769.0,353269.0,354756.0,356220.0,357667.0,359114.0,360533.0,307384.0,310207.0,312820.0,315250.0,317504.0,319560.0,321443.0,323187.0,324784.0,326263.0,327644.0,328927.0,330135.0,331275.0,332343.0,333334.0,334290.0,335271.0,336308.0,337386.0,338489.0,339578.0,340674.0,341828.0,343069.0,344392.0,345767.0,347216.0,348706.0,350230.0,351769.0,353269.0,354756.0,356220.0,357667.0,359114.0,360533.0
2,"Dickson County, Tennessee",21118,22540.0,21439.666667,21743.333333,22032.000000,22308.666667,22571.333333,22820.000000,23057.666667,23284.333333,23501.000000,23709.666667,23911.333333,24105.000000,24294.666667,24477.333333,24657.000000,24829.666667,25001.333333,25173.000000,25349.666667,25530.333333,25712.000000,25894.666667,26077.333333,26265.000000,26458.666667,26659.333333,26865.0,27023.0,27186.0,27351.0,27519.0,27684.0,27849.0,28013.0,28174.0,28337.0,28499.0,22809.0,23060.0,23296.0,23520.0,23730.0,23926.0,24111.0,24285.0,24449.0,24605.0,24754.0,24895.0,25032.0,25162.0,25289.0,25409.0,25528.0,25647.0,25771.0,25899.0,26028.0,26158.0,26288.0,26423.0,26564.0,26712.0,26865.0,27023.0,27186.0,27351.0,27519.0,27684.0,27849.0,28013.0,28174.0,28337.0,28499.0
3,"Houston County, Tennessee",3106,3578.0,3141.481481,3173.962963,3204.444444,3233.925926,3261.407407,3286.888889,3310.370370,3333.851852,3355.333333,3376.814815,3397.296296,3417.777778,3438.259259,3458.740741,3478.222222,3497.703704,3517.185185,3537.666667,3559.148148,3580.629630,3602.111111,3624.592593,3647.074074,3670.555556,3694.037037,3718.518519,3744.0,3752.0,3760.0,3769.0,3777.0,3785.0,3792.0,3798.0,3804.0,3810.0,3815.0,3596.0,3611.0,3624.0,3636.0,3646.0,3654.0,3660.0,3666.0,3670.0,3674.0,3677.0,3680.0,3683.0,3686.0,3688.0,3690.0,3692.0,3695.0,3699.0,3703.0,3707.0,3712.0,3717.0,3723.0,3729.0,3736.0,3744.0,3752.0,3760.0,3769.0,3777.0,3785.0,3792.0,3798.0,3804.0,3810.0,3815.0
4,"Humphreys County, Tennessee",6831,8222.0,6924.518519,7

In [504]:
data.to_csv('../data/smoothedhhs_2023Census.csv', index = False)

In [505]:
hhsize = hhsize.loc[hhsize['Year'] != '2022']
hhsize.head()

,NAME,Year,personsperhousehold
33,"Cheatham County, Tennessee",2023,2.46
34,"Cheatham County, Tennessee",2024,2.45
35,"Cheatham County, Tennessee",2025,2.45
36,"Cheatham County, Tennessee",2026,2.44
37,"Cheatham County, Tennessee",2027,2.44


In [506]:
hhs.head()

,NAME,Year,households
33,"Cheatham County, Tennessee",2023,16950.0
34,"Cheatham County, Tennessee",2024,17109.0
35,"Cheatham County, Tennessee",2025,17257.0
36,"Cheatham County, Tennessee",2026,17396.0
37,"Cheatham County, Tennessee",2027,17528.0


In [507]:
hhpopwp = hhsize.merge(hhs, on = ['NAME', 'Year'])
hhpopwp.head()

,NAME,Year,personsperhousehold,households
0,"Cheatham County, Tennessee",2023,2.46,16950.0
1,"Cheatham County, Tennessee",2024,2.45,17109.0
2,"Cheatham County, Tennessee",2025,2.45,17257.0
3,"Cheatham County, Tennessee",2026,2.44,17396.0
4,"Cheatham County, Tennessee",2027,2.44,17528.0


In [508]:
hhpopwp['Household Population'] = hhpopwp['personsperhousehold'] * hhpopwp['households']
hhpopwp = hhpopwp[['NAME', 'Year', 'Household Population']]

In [509]:
hhpopwp.head()

,NAME,Year,Household Population
0,"Cheatham County, Tennessee",2023,41697.00
1,"Cheatham County, Tennessee",2024,41917.05
2,"Cheatham County, Tennessee",2025,42279.65
3,"Cheatham County, Tennessee",2026,42446.24
4,"Cheatham County, Tennessee",2027,42768.32


In [510]:
hhpop = pd.DataFrame(list(hhpop.items()), columns=['NAME', '2023 Census'])
hhpop.head()

,NAME,2023 Census
0,"Cheatham County, Tennessee",41924
1,"Davidson County, Tennessee",688602
2,"Dickson County, Tennessee",55962
3,"Houston County, Tennessee",8262
4,"Humphreys County, Tennessee",18784


In [511]:
wide_df = hhpopwp.pivot_table(index='NAME', columns='Year', values='Household Population')
wide_df = wide_df.reset_index().rename_axis(None, axis=1)
wide_df = wide_df.rename(columns = {'2023': '2023 WP'})

In [512]:
merged_df = pd.merge(wide_df, hhpop[['NAME', '2023 Census']], on = 'NAME', how = 'inner')
column_order = ['NAME', '2023 Census'] + [col for col in merged_df.columns if col not in ['NAME', '2023 Census']]
merged_df = merged_df[column_order]

In [513]:
non_adj = merged_df
non_adj = non_adj.drop(columns = ['2023 Census', '2023 WP']).set_index('NAME')
non_adj = non_adj.add_suffix(" Non-Adjusted").reset_index()

In [514]:
years = list(merged_df.columns)
years.remove('NAME')
years.remove('2023 Census')
years.remove('2023 WP')
years.remove('2050')

In [515]:
def apply_formula(row):
    # Assuming '2023' is the base year for census and WP, and '2050' is the last year
    base_year = 2023
    final_year = 2050

    # Calculate the denominator part of the formula
    denominator = final_year - base_year
    
    # If denominator is zero, return 0 to avoid division by zero error
    if denominator == 0:
        return 0
    # Loop through the years between 2024 and 2049 (inclusive)
    for year in range(base_year + 1, final_year):
        # Calculate the adjusted value for the current year
        adjusted_value = row[f'{year}'] - (row[f'{base_year} WP'] - row[f'{base_year} Census']) / denominator * (final_year - int(year))
       # Update the value for the current year
        row[f'{year}'] = adjusted_value
    
    return row

# Apply the formula to each row in the DataFrame
merged_df = merged_df.apply(apply_formula, axis=1)

In [516]:
data = merged_df.merge(non_adj, on = 'NAME')
data.head()

,NAME,2023 Census,2023 WP,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050,2051,2052,2053,2054,2055,2056,2057,2058,2059,2060,2024 Non-Adjusted,2025 Non-Adjusted,2026 Non-Adjusted,2027 Non-Adjusted,2028 Non-Adjusted,2029 Non-Adjusted,2030 Non-Adjusted,2031 Non-Adjusted,2032 Non-Adjusted,2033 Non-Adjusted,2034 Non-Adjusted,2035 Non-Adjusted,2036 Non-Adjusted,2037 Non-Adjusted,2038 Non-Adjusted,2039 Non-Adjusted,2040 Non-Adjusted,2041 Non-Adjusted,2042 Non-Adjusted,2043 Non-Adjusted,2044 Non-Adjusted,2045 Non-Adjusted,2046 Non-Adjusted,2047 Non-Adjusted,2048 Non-Adjusted,2049 Non-Adjusted,2050 Non-Adjusted,2051 Non-Adjusted,2052 Non-Adjusted,2053 Non-Adjusted,2054 Non-Adjusted,2055 Non-Adjusted,2056 Non-Adjusted,2057 Non-Adjusted,2058 Non-Adjusted,2059 Non-Adjusted,2060 Non-Adjusted
0,"Cheatham County, Tennessee",41924,41697.00,42135.642593,42489.835185,42648.017778,42961.690370,43084.182963,43362.515556,43623.838148,43873.010741,44110.033333,44342.195926,44567.068519,44789.511111,45009.523704,45231.966296,45454.408889,45857.121481,46078.074074,46308.786667,46546.819259,46797.051852,47054.604444,47314.597037,47577.029630,47846.782222,48128.734815,48422.887407,48724.36,48840.57,49161.33,49486.95,49812.57,50130.90,50236.78,50544.12,50844.20,51141.86,51429.84,41917.05,42279.65,42446.24,42768.32,42899.22,43185.96,43455.69,43713.27,43958.70,44199.27,44432.55,44663.40,44891.82,45122.67,45353.52,45764.64,45994.00,46233.12,46479.56,46738.20,47004.16,47272.56,47543.40,47821.56,48111.92,48414.48,48724.36,48840.57,49161.33,49486.95,49812.57,50130.90,50236.78,50544.12,50844.20,51141.86,51429.84
1,"Davidson County, Tennessee",688602,687751.90,692432.614815,695650.809630,701472.444444,703731.659259,708726.594074,713279.988889,717447.593704,721305.228519,724835.053333,728101.738148,731149.882963,733979.487778,739943.192593,742465.307407,744826.142222,747014.497037,749124.451852,754643.116667,756944.881481,759338.896296,761789.161111,764207.925926,766642.440741,769207.455556,771968.220370,774913.485185,777975.75,781236.00,784588.50,784515.20,787962.56,791322.56,794653.44,797932.80,801174.08,804415.36,807593.92,691614.00,694863.68,700716.80,703007.50,708033.92,712618.80,716817.89,720707.01,724268.32,727566.49,730646.12,733507.21,739502.40,742056.00,744448.32,746668.16,748809.60,754359.75,756693.00,759118.50,761600.25,764050.50,766516.50,769113.00,771905.25,774882.00,777975.75,781236.00,784588.50,784515.20,787962.56,791322.56,794653.44,797932.80,801174.08,804415.36,807593.92
2,"Dickson County, Tennessee",55962,55448.40,56376.627778,56741.955556,57298.773333,57591.111111,58082.388889,58539.646667,58970.174444,59373.972222,59753.470000,60113.527778,60456.575556,60780.183333,61094.071111,61390.948889,61680.536667,62207.204444,62478.542222,62749.880000,63033.417778,63326.715556,63622.453333,63920.631111,64218.808889,64529.186667,64854.204444,65196.302222,65550.60,65665.89,66061.98,66462.93,66871.17,67272.12,67394.58,67791.46,68181.08,68575.54,68967.58,55882.05,56266.40,56842.24,57153.60,57663.90,58140.18,58589.73,59012.55,59411.07,59790.15,60152.22,60494.85,60827.76,61143.66,61452.27,61997.96,62288.32,62578.68,62881.24,63193.56,63508.32,63825.52,64142.72,64472.12,64816.16,65177.28,65550.60,65665.89,66061.98,66462.93,66871.17,67272.12,67394.58,67791.46,68181.08,68575.54,68967.58
3,"Houston County, Tennessee",8262,8050.50,8258.706667,8284.473333,8269.520000,8288.446667,8302.913333,8312.920000,8318.466667,8287.353333,8325.100000,8326.186667,8325.043333,8323.900000,8322.756667,8321.613333,8318.240000,8351.766667,8348.413333,8347.300000,8348.426667,8386.583333,8387.750000,8391.166667,8394.583333,8400.250000,8405.916667,8413.833333,8424.00,8442.00,8422.40,8442.56,8460.48,8478.40,8494.08,8507.52,8520.96,8534.40,8545.60,8055.04,8088.64,8081.52,8108.28,8130.58,8148.42,8161.80,8138.52,8184.10,8193.02,8199.71,8206.40,8213.09,8219.78,8224.24,8265.60,8270.08,8276.80,8285.76,8331.

In [517]:
hhpop = merged_df

In [518]:
data.to_csv('../data/smoothedhhpop_2023Census.csv', index = False)

In [519]:
pep = pd.read_csv('../data/2023_CountyPEP.csv')
pep = pep.rename(columns = {'2023': '2023 Census'})

In [520]:
conn = sq.connect('../../Data-Pipelines/Outputs/WoodsandPooleandAffiliated.db')
sql_query = pd.read_sql('SELECT * FROM [WPPopulations_2023Base]', conn)
pop = pd.DataFrame(sql_query)
pop = pop[['NAME', 'Year', 'totalpop']]
pop = pop.loc[pop['Year'] != 'None']

In [521]:
pop['Year'] = pop['Year'].astype(int)
pop = pop[pop['Year'] > 2022]
pop['Year'] = pop['Year'].astype(str)

In [522]:
wide_df = pop.pivot_table(index='NAME', columns='Year', values='totalpop')
wide_df = wide_df.reset_index().rename_axis(None, axis=1)

In [523]:
wide_df = wide_df.rename(columns = {'2023': '2023 WP'})
merged_df = pd.merge(wide_df, pep[['NAME', '2023 Census']], on = 'NAME', how = 'inner')
column_order = ['NAME', '2023 Census'] + [col for col in merged_df.columns if col not in ['NAME', '2023 Census']]
merged_df = merged_df[column_order]

In [524]:
non_adj = merged_df
non_adj = non_adj.drop(columns = ['2023 Census', '2023 WP']).set_index('NAME')
non_adj = non_adj.add_suffix(" Non-Adjusted").reset_index()

In [525]:
years = list(merged_df.columns)
years.remove('NAME')
years.remove('2023 Census')
years.remove('2023 WP')
years.remove('2050')

In [526]:
def apply_formula(row):
    # Assuming '2023' is the base year for census and WP, and '2050' is the last year
    base_year = 2023
    final_year = 2050

    # Calculate the denominator part of the formula
    denominator = final_year - base_year
    
    # If denominator is zero, return 0 to avoid division by zero error
    if denominator == 0:
        return 0
    # Loop through the years between 2024 and 2049 (inclusive)
    for year in range(base_year + 1, final_year):
        # Calculate the adjusted value for the current year
        adjusted_value = row[f'{year}'] - (row[f'{base_year} WP'] - row[f'{base_year} Census']) / denominator * (final_year - int(year))
       # Update the value for the current year
        row[f'{year}'] = adjusted_value
    
    return row

# Apply the formula to each row in the DataFrame
merged_df = merged_df.apply(apply_formula, axis=1)

In [527]:
data = merged_df.merge(non_adj, on = 'NAME')

In [528]:
totalpop = merged_df

In [529]:
data.to_csv('../data/smoothedtotalpop_2023PEP.csv', index = False)

In [530]:
totalpop.head()

,NAME,2023 Census,2023 WP,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050
0,"Cheatham County, Tennessee",42254,42068.0,42485.111111,42718.222222,42952.333333,43188.444444,43424.555556,43663.666667,43903.777778,44144.888889,44388.000000,44632.111111,44877.222222,45124.333333,45372.444444,45622.555556,45873.666667,46126.777778,46380.888889,46636.000000,46893.111111,47152.222222,47412.333333,47673.444444,47936.555556,48201.666667,48467.777778,48735.888889,49005.0
1,"Davidson County, Tennessee",712334,712937.0,717093.333333,721781.666667,726380.000000,730877.333333,735267.666667,739554.000000,743746.333333,747839.666667,751813.000000,755679.333333,759419.666667,763026.000000,766485.333333,769799.666667,772994.000000,776078.333333,779073.666667,781998.000000,784860.333333,787676.666667,790456.000000,793219.333333,795972.666667,798728.000000,801505.333333,804309.666667,807153.0
2,"Dickson County, Tennessee",56729,56221.0,57166.185185,57599.370370,58027.555556,58448.740741,58860.925926,59267.111111,59667.296296,60060.481481,60444.666667,60821.851852,61189.037037,61547.222222,61894.407407,62231.592593,62558.777778,62878.962963,63193.148148,63501.333333,63805.518519,64106.703704,64406.888889,64705.074074,65004.259259,65303.444444,65605.629630,65911.814815,66220.0
3,"Houston County, Tennessee",8393,8231.0,8400.000000,8407.000000,8413.000000,8420.000000,8427.000000,8434.000000,8440.000000,8447.000000,8454.000000,8461.000000,8468.000000,8473.000000,8480.000000,8487.000000,8494.000000,8501.000000,8508.000000,8515.000000,8522.000000,8529.000000,8536.000000,8543.000000,8550.000000,8557.000000,8564.000000,8571.000000,8579.0
4,"Humphreys County, Tennessee",19209,19139.0,19238.407407,19268.814815,19300.222222,19330.629630,19360.037037,19391.444444,19421.851852,19452.259259,19482.666667,19514.074074,19545.481481,19575.888889,19606.296296,19637.703704,19669.111111,19699.518519,19730.925926,19762.333333,19793.740741,19825.148148,19856.555556,19887.962963,19919.370370,19950.777778,19983.185185,20014.592593,20046.0


In [531]:
hhpop.head()

,NAME,2023 Census,2023 WP,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050,2051,2052,2053,2054,2055,2056,2057,2058,2059,2060
0,"Cheatham County, Tennessee",41924,41697.00,42135.642593,42489.835185,42648.017778,42961.690370,43084.182963,43362.515556,43623.838148,43873.010741,44110.033333,44342.195926,44567.068519,44789.511111,45009.523704,45231.966296,45454.408889,45857.121481,46078.074074,46308.786667,46546.819259,46797.051852,47054.604444,47314.597037,47577.029630,47846.782222,48128.734815,48422.887407,48724.36,48840.57,49161.33,49486.95,49812.57,50130.90,50236.78,50544.12,50844.20,51141.86,51429.84
1,"Davidson County, Tennessee",688602,687751.90,692432.614815,695650.809630,701472.444444,703731.659259,708726.594074,713279.988889,717447.593704,721305.228519,724835.053333,728101.738148,731149.882963,733979.487778,739943.192593,742465.307407,744826.142222,747014.497037,749124.451852,754643.116667,756944.881481,759338.896296,761789.161111,764207.925926,766642.440741,769207.455556,771968.220370,774913.485185,777975.75,781236.00,784588.50,784515.20,787962.56,791322.56,794653.44,797932.80,801174.08,804415.36,807593.92
2,"Dickson County, Tennessee",55962,55448.40,56376.627778,56741.955556,57298.773333,57591.111111,58082.388889,58539.646667,58970.174444,59373.972222,59753.470000,60113.527778,60456.575556,60780.183333,61094.071111,61390.948889,61680.536667,62207.204444,62478.542222,62749.880000,63033.417778,63326.715556,63622.453333,63920.631111,64218.808889,64529.186667,64854.204444,65196.302222,65550.60,65665.89,66061.98,66462.93,66871.17,67272.12,67394.58,67791.46,68181.08,68575.54,68967.58
3,"Houston County, Tennessee",8262,8050.50,8258.706667,8284.473333,8269.520000,8288.446667,8302.913333,8312.920000,8318.466667,8287.353333,8325.100000,8326.186667,8325.043333,8323.900000,8322.756667,8321.613333,8318.240000,8351.766667,8348.413333,8347.300000,8348.426667,8386.583333,8387.750000,8391.166667,8394.583333,8400.250000,8405.916667,8413.833333,8424.00,8442.00,8422.40,8442.56,8460.48,8478.40,8494.08,8507.52,8520.96,8534.40,8545.60
4,"Humphreys County, Tennessee",18784,18828.38,18881.823704,18882.907407,18957.511111,18941.664815,18997.788519,19047.102222,19087.335926,19123.029630,19154.183333,19178.527037,19200.600741,19220.404444,19325.068148,19347.231852,19367.115556,19382.439259,19400.042963,19505.166667,19529.710370,19561.124074,19592.537778,19621.661481,19653.075185,19691.358889,19729.642593,19770.216296,19817.66,19863.46,19911.55,19959.64,19918.08,19963.68,20004.72,20043.48,20077.68,20111.88,20143.80


In [532]:
totalpop = totalpop.drop(columns = '2023 WP')
totalpop = totalpop.melt(id_vars = 'NAME', var_name = 'Year', value_name = 'Population')
hhpop = hhpop.drop(columns = '2023 WP')
hhpop = hhpop.melt(id_vars = 'NAME', var_name = 'Year', value_name = 'Household Population')

In [533]:
data = totalpop.merge(hhpop, on = ['NAME', 'Year'])

In [534]:
data.head()

,NAME,Year,Population,Household Population
0,"Cheatham County, Tennessee",2023 Census,42254.0,41924.0
1,"Davidson County, Tennessee",2023 Census,712334.0,688602.0
2,"Dickson County, Tennessee",2023 Census,56729.0,55962.0
3,"Houston County, Tennessee",2023 Census,8393.0,8262.0
4,"Humphreys County, Tennessee",2023 Census,19209.0,18784.0


In [535]:
data['Group Quarters'] = data['Population'] - data['Household Population']

In [536]:
data = data[['NAME', 'Year', 'Group Quarters']]

In [537]:
data.head()

,NAME,Year,Group Quarters
0,"Cheatham County, Tennessee",2023 Census,330.0
1,"Davidson County, Tennessee",2023 Census,23732.0
2,"Dickson County, Tennessee",2023 Census,767.0
3,"Houston County, Tennessee",2023 Census,131.0
4,"Humphreys County, Tennessee",2023 Census,425.0


In [538]:
gqbase = pd.DataFrame(list(gq.items()), columns=['NAME', '2023 Census'])
gqbase.head()

,NAME,2023 Census
0,"Cheatham County, Tennessee",330
1,"Davidson County, Tennessee",23732
2,"Dickson County, Tennessee",767
3,"Houston County, Tennessee",131
4,"Humphreys County, Tennessee",425


In [539]:
wide_df = data.pivot_table(index='NAME', columns='Year', values='Group Quarters')
wide_df = wide_df.reset_index().rename_axis(None, axis=1)
wide_df = wide_df.rename(columns = {'2023 Census': '2023 WP'})

In [540]:
wide_df.head()

,NAME,2023 WP,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050
0,"Cheatham County, Tennessee",330.0,349.468519,228.387037,304.315556,226.754074,340.372593,301.151111,279.939630,271.878148,277.966667,289.915185,310.153704,334.822222,362.920741,390.589259,419.257778,269.656296,302.814815,327.213333,346.291852,355.170370,357.728889,358.847407,359.525926,354.884444,339.042963,313.001481,280.64
1,"Davidson County, Tennessee",23732.0,24660.718519,26130.857037,24907.555556,27145.674074,26541.072593,26274.011111,26298.739630,26534.438148,26977.946667,27577.595185,28269.783704,29046.512222,26542.140741,27334.359259,28167.857778,29063.836296,29949.214815,27354.883333,27915.451852,28337.770370,28666.838889,29011.407407,29330.225926,29520.544444,29537.112963,29396.181481,29177.25
2,"Dickson County, Tennessee",767.0,789.557407,857.414815,728.782222,857.629630,778.537037,727.464444,697.121852,686.509259,691.196667,708.324074,732.461481,767.038889,800.336296,840.643704,878.241111,671.758519,714.605926,751.453333,772.100741,779.988148,784.435556,784.442963,785.450370,774.257778,751.425185,715.512593,669.40
3,"Houston County, Tennessee",131.0,141.293333,122.526667,143.480000,131.553333,124.086667,121.080000,121.533333,159.646667,128.900000,134.813333,142.956667,149.100000,157.243333,165.386667,175.760000,149.233333,159.586667,167.700000,173.573333,142.416667,148.250000,151.833333,155.416667,156.750000,158.083333,157.166667,155.00
4,"Humphreys County, Tennessee",425.0,356.583704,385.907407,342.711111,388.964815,362.248519,344.342222,334.515926,329.229630,328.483333,335.547037,344.880741,355.484444,281.228148,290.471852,301.995556,317.079259,330.882963,257.166667,264.030370,264.024074,264.017778,266.301481,266.295185,259.418889,253.542593,244.376296,228.34


In [541]:
#wide_df = wide_df.rename(columns = {'2023': '2023 WP'})
merged_df = pd.merge(wide_df, gqbase, on = 'NAME', how = 'inner')
column_order = ['NAME', '2023 Census'] + [col for col in merged_df.columns if col not in ['NAME', '2023 Census']]
merged_df = merged_df[column_order]

In [542]:
non_adj = merged_df
non_adj = non_adj.drop(columns = ['2023 Census', '2023 WP']).set_index('NAME')
non_adj = non_adj.add_suffix(" Non-Adjusted").reset_index()

In [543]:
years = list(merged_df.columns)
years.remove('NAME')
years.remove('2023 Census')
years.remove('2023 WP')
years.remove('2050')

In [544]:
def apply_formula(row):
    # Assuming '2023' is the base year for census and WP, and '2050' is the last year
    base_year = 2023
    final_year = 2050

    # Calculate the denominator part of the formula
    denominator = final_year - base_year
    
    # If denominator is zero, return 0 to avoid division by zero error
    if denominator == 0:
        return 0
    # Loop through the years between 2024 and 2049 (inclusive)
    for year in range(base_year + 1, final_year):
        # Calculate the adjusted value for the current year
        adjusted_value = row[f'{year}'] - (row[f'{base_year} WP'] - row[f'{base_year} Census']) / denominator * (final_year - int(year))
       # Update the value for the current year
        row[f'{year}'] = adjusted_value
    
    return row

# Apply the formula to each row in the DataFrame
merged_df = merged_df.apply(apply_formula, axis=1)

In [545]:
data = merged_df.merge(non_adj, on = 'NAME')

In [546]:
data.head()

,NAME,2023 Census,2023 WP,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050,2024 Non-Adjusted,2025 Non-Adjusted,2026 Non-Adjusted,2027 Non-Adjusted,2028 Non-Adjusted,2029 Non-Adjusted,2030 Non-Adjusted,2031 Non-Adjusted,2032 Non-Adjusted,2033 Non-Adjusted,2034 Non-Adjusted,2035 Non-Adjusted,2036 Non-Adjusted,2037 Non-Adjusted,2038 Non-Adjusted,2039 Non-Adjusted,2040 Non-Adjusted,2041 Non-Adjusted,2042 Non-Adjusted,2043 Non-Adjusted,2044 Non-Adjusted,2045 Non-Adjusted,2046 Non-Adjusted,2047 Non-Adjusted,2048 Non-Adjusted,2049 Non-Adjusted,2050 Non-Adjusted
0,"Cheatham County, Tennessee",330,330.0,349.468519,228.387037,304.315556,226.754074,340.372593,301.151111,279.939630,271.878148,277.966667,289.915185,310.153704,334.822222,362.920741,390.589259,419.257778,269.656296,302.814815,327.213333,346.291852,355.170370,357.728889,358.847407,359.525926,354.884444,339.042963,313.001481,280.64,349.468519,228.387037,304.315556,226.754074,340.372593,301.151111,279.939630,271.878148,277.966667,289.915185,310.153704,334.822222,362.920741,390.589259,419.257778,269.656296,302.814815,327.213333,346.291852,355.170370,357.728889,358.847407,359.525926,354.884444,339.042963,313.001481,280.64
1,"Davidson County, Tennessee",23732,23732.0,24660.718519,26130.857037,24907.555556,27145.674074,26541.072593,26274.011111,26298.739630,26534.438148,26977.946667,27577.595185,28269.783704,29046.512222,26542.140741,27334.359259,28167.857778,29063.836296,29949.214815,27354.883333,27915.451852,28337.770370,28666.838889,29011.407407,29330.225926,29520.544444,29537.112963,29396.181481,29177.25,24660.718519,26130.857037,24907.555556,27145.674074,26541.072593,26274.011111,26298.739630,26534.438148,26977.946667,27577.595185,28269.783704,29046.512222,26542.140741,27334.359259,28167.857778,29063.836296,29949.214815,27354.883333,27915.451852,28337.770370,28666.838889,29011.407407,29330.225926,29520.544444,29537.112963,29396.181481,29177.25
2,"Dickson County, Tennessee",767,767.0,789.557407,857.414815,728.782222,857.629630,778.537037,727.464444,697.121852,686.509259,691.196667,708.324074,732.461481,767.038889,800.336296,840.643704,878.241111,671.758519,714.605926,751.453333,772.100741,779.988148,784.435556,784.442963,785.450370,774.257778,751.425185,715.512593,669.40,789.557407,857.414815,728.782222,857.629630,778.537037,727.464444,697.121852,686.509259,691.196667,708.324074,732.461481,767.038889,800.336296,840.643704,878.241111,671.758519,714.605926,751.453333,772.100741,779.988148,784.435556,784.442963,785.450370,774.257778,751.425185,715.512593,669.40
3,"Houston County, Tennessee",131,131.0,141.293333,122.526667,143.480000,131.553333,124.086667,121.080000,121.533333,159.646667,128.900000,134.813333,142.956667,149.100000,157.243333,165.386667,175.760000,149.233333,159.586667,167.700000,173.573333,142.416667,148.250000,151.833333,155.416667,156.750000,158.083333,157.166667,155.00,141.293333,122.526667,143.480000,131.553333,124.086667,121.080000,121.533333,159.646667,128.900000,134.813333,142.956667,149.100000,157.243333,165.386667,175.760000,149.233333,159.586667,167.700000,173.573333,142.416667,148.250000,151.833333,155.416667,156.750000,158.083333,157.166667,155.00
4,"Humphreys County, Tennessee",425,425.0,356.583704,385.907407,342.711111,388.964815,362.248519,344.342222,334.515926,329.229630,328.483333,335.547037,344.880741,355.484444,281.228148,290.471852,301.995556,317.079259,330.882963,257.166667,264.030370,264.024074,264.017778,266.301481,266.295185,259.418889,253.542593,244.376296,228.34,356.583704,385.907407,342.711111,388.964815,362.248519,344.342222,334.515926,329.229630,328.483333,335.547037,344.880741,355.484444,281.228148,290.471852,301.995556,317.079259,330.882963,257.166667,264.030370,264.024074,264.017778,266.301481,266.295185,259.418889,253.542593,244.376296,228.34


In [547]:
data.to_csv('../data/smoothedgqpop_2023PEP.csv', index = False)